## <center> <font color='purple'> Dynamic Ensemble Machine Learning Models (Static Ensemble ML Pool)</font></center> 
#### <center>Firuz Juraev (Sungkyunkwan Unniversity)</center>

### <font color='green'> Libraries 

#### <font color='blue'> Basic Libraries 

In [1]:
import pandas as pd
import numpy as np 
from numpy import mean
from numpy import std
import random
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

#### <font color='blue'> Single ML Models Libraries 

In [2]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

#### <font color='blue'> Static ML Models Libraries 

In [3]:
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

#### <font color='blue'> DES Libraries 

In [4]:
from deslib.des import DESP
from deslib.des import KNORAE
from deslib.des import KNORAU
from deslib.des import METADES
from deslib.des import DESKNN # new 
from deslib.des import KNOP # new 

#### <font color='blue'> DCS Libraries 

In [5]:
from deslib.dcs import MCB

#### <font color='blue'> Processing Libraries 

In [6]:
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix, confusion_matrix

from sklearn.metrics import (accuracy_score,
                             precision_score,
                             recall_score, 
                             f1_score,
                             roc_auc_score, 
                             auc)
from sklearn.metrics import roc_curve, roc_auc_score 
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

### <font color='green'> Load Dataset 

In [7]:
train_dataset_upsampled = pd.read_csv("Data/Resampled_neonates_train_data_4.csv")
test_dataset_upsampled = pd.read_csv("Data/Resampled_neonates_test_data_4.csv")

X_train = train_dataset_upsampled.drop(["DEAD"], axis=1) 
y_train = train_dataset_upsampled["DEAD"]

X_test = test_dataset_upsampled.drop(["DEAD"], axis=1) 
y_test = test_dataset_upsampled["DEAD"]

In [8]:
columns = X_train.columns.to_list()

### <font color='green'> Data Normalization

In [9]:
Min_max_scaler = MinMaxScaler().fit(X_train)

## Scaling 
X_train_mm_scaled = Min_max_scaler.transform(X_train)
X_test_mm_scaled = Min_max_scaler.transform(X_test)

## Numpy Array to DataFrame 
df_train_mm_scaled = pd.DataFrame(X_train_mm_scaled, columns = columns)
df_test_mm_scaled = pd.DataFrame(X_test_mm_scaled, columns = columns)

### <font color='green'> Feature Selection

In [10]:
tain_mm_scaled_df = df_train_mm_scaled[:]
tain_mm_scaled_df["DEAD"] = y_train 

test_mm_scaled_df = df_test_mm_scaled[:] 
test_mm_scaled_df["DEAD"] = y_test


importances = mutual_info_classif(df_train_mm_scaled, y_train)
feat_importance = pd.Series(importances, tain_mm_scaled_df.columns[0:len(tain_mm_scaled_df.columns)-1])
    
feat_importance = feat_importance.sort_values(ascending=False)
    
selected_features = feat_importance[:30]
selected_features_list_mm_scaled = selected_features.index.to_list()


tain_mm_scaled_df[selected_features_list_mm_scaled].head(2)

,temperature_mean,respRate_std,respRate_var,skinTemperature_std,skinTemperature_var,heartRate_std,heartRate_var,sao2_std,sao2_var,bpCuffMean_var,...,BIRTH_WEIGHT,bpCuffSystolic_mean,bpCuffDiastolic_mean,glucometer_mean,temperature_std,sao2_mean,temperature_var,bpCuffMean_mean,PLATELET,D10W_MEAN
0,0.775318,0.422245,0.178291,0.134078,0.017977,0.22911,0.057937,0.097817,0.009568,0.053294,...,0.264267,0.602649,0.457143,0.312593,0.125281,0.881455,0.015695,0.310696,0.281330,0.151138
1,0.709668,0.583964,0.341014,0.086672,0.007512,0.28258,0.086102,0.080938,0.006551,0.020317,...,0.437819,0.644907,0.479330,0.272551,0.061399,0.926056,0.003770,0.351928,0.392157,0.196784


In [11]:
X_train_mm = df_train_mm_scaled[selected_features_list_mm_scaled][:]
X_test_mm = df_test_mm_scaled[selected_features_list_mm_scaled][:]

### <font color='purple'> Hold-out Test (With Single ML) - (+FS, +HO)

In [12]:
def hold_out_single_ML(): 
    rng = np.random.RandomState(42) 
    X_train, X_dsel, y_train_en, y_dsel = train_test_split(X_train_mm, y_train, test_size=0.40, random_state=rng)
    
    model_ada = AdaBoostClassifier(n_estimators=250, learning_rate=0.01)
    model_gb = GradientBoostingClassifier(learning_rate = 0.005, n_estimators=120, max_depth=4)
    model_rf  = RandomForestClassifier(criterion='gini', n_estimators=100, max_depth=2)
    model_cat = MLPClassifier(solver='adam', max_iter=11, verbose=10,learning_rate_init=.003)
    model_lgb = LGBMClassifier(max_depth=1, n_estimators=150, objective="binary")
    
    model_ada.fit(X_train, y_train_en)
    model_gb.fit(X_train, y_train_en)
    model_rf.fit(X_train, y_train_en)
    model_cat.fit(X_train, y_train_en)
    model_lgb.fit(X_train, y_train_en) 
    
    classifiers_names = ["AdaBoost","GradientBoosting", "RandomForest", "CatBoost", "LGBM"]
    
    pool_classifiers = [model_ada, 
                        model_gb,
                        model_rf, 
                        model_cat, 
                        model_lgb] 
    
    c_acc_list = [] 
    
    for cls in pool_classifiers:
        y_preds_c = cls.predict(X_test_mm) 
        c_acc_list.append(accuracy_score(y_test, y_preds_c))           
        
        
    classifiers_results =  {'name': classifiers_names, 
                            'accuracy': c_acc_list}
    
    clsDF = pd.DataFrame.from_dict(classifiers_results)
        
    # DES STARTS
    
    knorau = KNORAU(pool_classifiers)
    kne = KNORAE(pool_classifiers)  
    metades = METADES(pool_classifiers)
    desknn = DESKNN(pool_classifiers)
    mcb = MCB(pool_classifiers)
    desp = DESP(pool_classifiers)
    knop = KNOP(pool_classifiers)

    fire_knorau = KNORAU(pool_classifiers, DFP=True, k=9) 
    fire_kne = KNORAE(pool_classifiers, DFP=True, k=9) 
    fire_metades = METADES(pool_classifiers, DFP=True, k=9) 
    fire_desknn = DESKNN(pool_classifiers, DFP=True, k=7) 
    fire_mcb = MCB(pool_classifiers, DFP=True, k=9)
    fire_desp = DESP(pool_classifiers, DFP=True, k=9)
    fire_knop = KNOP(pool_classifiers, DFP=True, k=15)
    
    ensemble_classifiers = [fire_knorau, knorau, fire_kne, kne, fire_metades, metades, fire_desknn, desknn, 
                            fire_mcb, mcb, fire_desp, desp, fire_knop, knop]
    
    ensemble_names = ["FIRE-KNORA-U", "KNORA-U", "FIRE-KNORA-E", "KNORA-E", "FIRE-METADES", "METADES",
                      "FIRE-DESKNN", "DESKNN", "FIRE-MCB", "MCB", "FIRE-DESP", "DESP", "FIRE-KNOP", "KNOP"]
    
    acc_list = [] 
    precision_list = [] 
    recall_list = []
    f1_lists = [] 
    auc_list = [] 
    fpr_list = [] 
    tpr_list = []
    
    for e_cls in ensemble_classifiers:
        e_cls.fit(X_dsel, y_dsel)
        y_preds = e_cls.predict(X_test_mm) 
        yproba = e_cls.predict_proba(X_test_mm)[::,1] 

        acc_list.append(accuracy_score(y_test, y_preds))
        precision_list.append(precision_score(y_test, y_preds))
        recall_list.append(recall_score(y_test, y_preds))
        fone = f1_score(y_test, y_preds)
        f1_lists.append(fone) 
        
        fpr, tpr, _ = roc_curve(y_test,  yproba)
        fpr_list.append(fpr)
        tpr_list.append(tpr)
        auc = roc_auc_score(y_test, yproba)
        auc_list.append(auc) 
    
    results =  {'name': ensemble_names, 
                'accuracy': acc_list, 
                'precision': precision_list, 
                'recall': recall_list, 'f1': f1_lists, 
                'auc': auc_list, 
                'tpr': tpr_list, 'fpr': fpr_list}
    
    df = pd.DataFrame.from_dict(results)
    
    return df, clsDF

In [13]:
results_data = []
classifier_results_data = []
for i in range(0, 10):
    result, cls_results = hold_out_single_ML()
    results_data.append(result)
    classifier_results_data.append(cls_results)
    

fireResultsDF = pd.concat(results_data)
classifiersResultsDF = pd.concat(classifier_results_data)

Iteration 1, loss = 0.62498536
Iteration 2, loss = 0.50960486
Iteration 3, loss = 0.42627843
Iteration 4, loss = 0.37754365
Iteration 5, loss = 0.34864824
Iteration 6, loss = 0.32836873
Iteration 7, loss = 0.31192767
Iteration 8, loss = 0.30036583
Iteration 9, loss = 0.29082125
Iteration 10, loss = 0.28203061
Iteration 11, loss = 0.27492753
Iteration 1, loss = 0.63349140
Iteration 2, loss = 0.52761270
Iteration 3, loss = 0.43899362
Iteration 4, loss = 0.38634076
Iteration 5, loss = 0.35371079
Iteration 6, loss = 0.33277689
Iteration 7, loss = 0.31683374
Iteration 8, loss = 0.30283511
Iteration 9, loss = 0.29404736
Iteration 10, loss = 0.28363160
Iteration 11, loss = 0.27643506
Iteration 1, loss = 0.62917275
Iteration 2, loss = 0.51317296
Iteration 3, loss = 0.42741426
Iteration 4, loss = 0.37615068
Iteration 5, loss = 0.35012577
Iteration 6, loss = 0.32783620
Iteration 7, loss = 0.31163178
Iteration 8, loss = 0.29883071
Iteration 9, loss = 0.28898215
Iteration 10, loss = 0.28233336
Ite

In [14]:
ensemble_names = ["FIRE-KNORA-U", "KNORA-U", "FIRE-KNORA-E", "KNORA-E", "FIRE-METADES", "METADES",
                  "FIRE-DESKNN", "DESKNN", "FIRE-MCB", "MCB", "FIRE-DESP", "DESP", "FIRE-KNOP", "KNOP"]

accuracy = [] 
accuracy_std =  [] 
precision = [] 
precision_std = [] 
recall = [] 
recall_std = [] 
f1_score = [] 
f1_std = []
auc_score = []
auc_std = []


for n in ensemble_names:
    accuracy.append(round(fireResultsDF[fireResultsDF.name == n].accuracy.mean(), 3))
    accuracy_std.append(round(fireResultsDF[fireResultsDF.name == n].accuracy.std(), 3))
    precision.append(round(fireResultsDF[fireResultsDF.name == n].precision.mean(), 3))
    precision_std.append(round(fireResultsDF[fireResultsDF.name == n].precision.std(), 3))
    recall.append(round(fireResultsDF[fireResultsDF.name == n].recall.mean(), 3))
    recall_std.append(round(fireResultsDF[fireResultsDF.name == n].recall.std(), 3))
    f1_score.append(round(fireResultsDF[fireResultsDF.name == n].f1.mean(), 3))
    f1_std.append(round(fireResultsDF[fireResultsDF.name == n].f1.std(), 3))
    auc_score.append(round(fireResultsDF[fireResultsDF.name == n].auc.mean(), 3))
    auc_std.append(round(fireResultsDF[fireResultsDF.name == n].auc.std(), 3))
    
final_results = {"method": ensemble_names, 
                     "accuracy": accuracy, 
                     "accuracy_std": accuracy_std,
                     "precision": precision, 
                     "precision_std": precision_std,
                     "recall": recall, 
                     "recall_std": recall_std,
                     "f1_score": f1_score, 
                     "f1_std": f1_std, 
                     "auc": auc_score, 
                     "auc_std": auc_std}

finalResultsDF = pd.DataFrame.from_dict(final_results)

In [21]:
finalResultsDF

,method,accuracy,accuracy_std,precision,precision_std,recall,recall_std,f1_score,f1_std,auc,auc_std
0,FIRE-KNORA-U,0.967,0.010,0.944,0.002,0.994,0.020,0.968,0.010,0.985,0.000
1,KNORA-U,0.964,0.010,0.938,0.002,0.994,0.020,0.965,0.010,0.985,0.000
2,FIRE-KNORA-E,0.975,0.010,0.959,0.001,0.994,0.020,0.976,0.011,0.989,0.000
3,KNORA-E,0.976,0.010,0.960,0.001,0.994,0.020,0.976,0.011,0.989,0.000
4,FIRE-METADES,0.910,0.016,0.968,0.002,0.849,0.031,0.904,0.018,0.954,0.025
5,METADES,0.906,0.010,0.966,0.002,0.842,0.021,0.900,0.012,0.943,0.022
6,FIRE-DESKNN,0.936,0.014,0.970,0.002,0.901,0.028,0.934,0.016,0.986,0.001
7,DESKNN,0.937,0.014,0.971,0.002,0.901,0.028,0.934,0.016,0.986,0.001
8,FIRE-MCB,0.952,0.005,0.955,0.004,0.949,0.010,0.952,0.005,0.975,0.002
9,MCB,0.954,0.003,0.955,0.003,0.952,0.006,0.953,0.003,0.975,0.003


In [16]:
classifiers_names = ["AdaBoost", "GradientBoosting", "RandomForest", "CatBoost", "LGBM"]

accuracy_list = []
accuracy_std_list = []
for n in classifiers_names:
    accuracy_list.append(classifiersResultsDF[classifiersResultsDF.name == n].accuracy.mean())
    accuracy_std_list.append(classifiersResultsDF[classifiersResultsDF.name == n].accuracy.std())

final_cls_results = {"classifier": classifiers_names, 
                     "accuracy": accuracy_list, 
                     "accuracy_std": accuracy_std_list}

finalClassifierResultsDF = pd.DataFrame.from_dict(final_cls_results)

In [19]:
finalClassifierResultsDF

,classifier,accuracy,accuracy_std
0,AdaBoost,0.950378,0.000000e+00
1,GradientBoosting,0.912082,1.170278e-16
2,RandomForest,0.958145,9.851521e-03
3,CatBoost,0.945038,1.201903e-02
4,LGBM,0.932039,2.340556e-16


In [22]:
finalResultsDF.describe().T

,count,mean,std,min,25%,50%,75%,max
accuracy,14.0,0.948143,0.024076,0.906,0.93625,0.9530,0.96775,0.976
accuracy_std,14.0,0.011000,0.004000,0.003,0.01000,0.0100,0.01400,0.018
precision,14.0,0.958214,0.010686,0.938,0.95050,0.9595,0.96750,0.971
precision_std,14.0,0.002071,0.000730,0.001,0.00200,0.0020,0.00200,0.004
recall,14.0,0.938286,0.059009,0.842,0.90100,0.9505,0.99400,0.994
recall_std,14.0,0.021929,0.007839,0.006,0.02000,0.0200,0.02775,0.036
f1_score,14.0,0.946714,0.026656,0.900,0.93400,0.9525,0.96875,0.976
f1_std,14.0,0.011929,0.004649,0.003,0.01000,0.0110,0.01600,0.019
auc,14.0,0.976714,0.015828,0.943,0.97500,0.9850,0.98750,0.989
auc_std,14.0,0.004643,0.008354,0.000,0.00000,0.0010,0.00275,0.025


In [23]:
finalResultsDF.to_csv("Results/static_pool_results/des_static_pool_5_models_results.csv", index=False)